# Interpolasi

In [37]:
import pandas as pd

# Load dataset varietas (Profile Ideal)
df_varietas = pd.read_excel('./dataset/split_data/varietas_iklim.xlsx')

# Load dataset lahan (Profile Alternatif)
df_lahan = pd.read_excel('./dataset/split_data/lahan_iklim.xlsx')

# Function utama untuk interpolasi
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Function untuk menghitung interpolasi berdasarkan jenis varietas
def hitung_skor_varietas(row, jenis_varietas):
    # Mengambil data varietas berdasarkan jenis_varietas
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]
    
    hasil = {
        'suhu': interpolasi(row['suhu_mean'], var_data['suhu_min'], var_data['suhu_max']),
        'curah_hujan': interpolasi(row['curah_hujan'], var_data['curah_hujan_min'], var_data['curah_hujan_max']),
        'kelembapan_udara': interpolasi(row['kelembapan_udara'], var_data['kelembapan_udara_min'], var_data['kelembapan_udara_max']),
        'penyinaran': interpolasi(row['penyinaran'], var_data['penyinaran_min'], var_data['penyinaran_max'])
    }
    
    return hasil

# penggunaan untuk varietas 
def hitung_skor_MM(row):
    return hitung_skor_varietas(row, 'MM')

def hitung_skor_MR(row):
    return hitung_skor_varietas(row, 'MR')

def hitung_skor_MK(row):
    return hitung_skor_varietas(row, 'MK')

def hitung_skor_MW(row):
    return hitung_skor_varietas(row, 'MW')

def hitung_skor_MT(row):
    return hitung_skor_varietas(row, 'MT')




# Bobot Utama

In [38]:
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

### BOBOT AHP Parameter Iklim

In [39]:
bobot_iklim = pd.read_excel('./dataset/bobot_ahp/parameter_iklim.xlsx')
bobot_iklim_dict = bobot_iklim.set_index('Label')['Weight'].to_dict()
bobot_iklim_dict

{'suhu': 0.2117957615233342,
 'curah_hujan': 0.5251582768815123,
 'kelembapan_udara': 0.05125020007181938,
 'penyinaran': 0.2117957615233342}

## Varietas MM

In [40]:
lahan_scores = df_lahan.apply(hitung_skor_MM, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_MM = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_MM

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor MM dengan bobot AHP)

In [41]:
skor_iklim_MM = lahan_iklim_MM.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_MM[kolom] = skor_iklim_MM[kolom] * bobot_iklim_dict[kolom]

skor_iklim_MM['total_skor'] = skor_iklim_MM[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_MM


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim MM dengan bobot AHP tanah

In [42]:

skor_total_iklim_MM = skor_iklim_MM.copy()
skor_total_iklim_MM['total_skor_iklim_MM'] = skor_total_iklim_MM['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_MM


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_MM
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas MR

In [43]:
lahan_scores = df_lahan.apply(hitung_skor_MR, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_MR = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_MR

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (perkalian skor MR dengan bobot AHP )

In [44]:
skor_iklim_MR = lahan_iklim_MR.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_MR[kolom] = skor_iklim_MR[kolom] * bobot_iklim_dict[kolom]

skor_iklim_MR['total_skor'] = skor_iklim_MR[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_MR


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim MR dengan bobot AHP tanah

In [45]:

skor_total_iklim_MR = skor_iklim_MR.copy()
skor_total_iklim_MR['total_skor_iklim_MR'] = skor_total_iklim_MR['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_MR


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_MR
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas MK

In [46]:
lahan_scores = df_lahan.apply(hitung_skor_MK, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_MK = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_MK

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor MK dengan bobot AHP)

In [47]:
skor_iklim_MK = lahan_iklim_MK.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_MK[kolom] = skor_iklim_MK[kolom] * bobot_iklim_dict[kolom]

skor_iklim_MK['total_skor'] = skor_iklim_MK[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_MK


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim MK dengan bobot AHP tanah

In [48]:

skor_total_iklim_MK = skor_iklim_MK.copy()
skor_total_iklim_MK['total_skor_iklim_MK'] = skor_total_iklim_MK['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_MK


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_MK
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas MW

In [49]:
lahan_scores = df_lahan.apply(hitung_skor_MW, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_MW = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_MW

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor MK dengan bobot AHP)

In [50]:
skor_iklim_MW = lahan_iklim_MW.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_MW[kolom] = skor_iklim_MW[kolom] * bobot_iklim_dict[kolom]

skor_iklim_MW['total_skor'] = skor_iklim_MW[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_MW


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim MW dengan bobot AHP tanah

In [51]:

skor_total_iklim_MW = skor_iklim_MW.copy()
skor_total_iklim_MW['total_skor_iklim_MW'] = skor_total_iklim_MW['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_MW


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_MW
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


## Varietas MT

In [52]:
lahan_scores = df_lahan.apply(hitung_skor_MT, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi
lahan_iklim_MT = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_iklim_MT

,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


### SUM (Perkalian skor MT dengan bobot AHP)

In [53]:
skor_iklim_MT = lahan_iklim_MT.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    skor_iklim_MT[kolom] = skor_iklim_MT[kolom] * bobot_iklim_dict[kolom]

skor_iklim_MT['total_skor'] = skor_iklim_MT[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

skor_iklim_MT


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


### Perkalian skor total iklim MT dengan bobot AHP tanah

In [54]:

skor_total_iklim_MT = skor_iklim_MT.copy()
skor_total_iklim_MT['total_skor_iklim_MT'] = skor_total_iklim_MT['total_skor'] * bobot_utama_dict['iklim']
skor_total_iklim_MT


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim_MT
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
